In [10]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]
colmul = mydb["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print("Día de la semana:",comp)
if (comp == '0') :#--------------------------------------------------------------cuidado
    esjuego=False
    esDLC=False
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor()

    mycursor.execute("SELECT * FROM transacciones_compra WHERE fecha_compra BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "compra"
    for t in transacciones :
   
    #"movimiento"
        precio_final = t[1]
        mycursor.execute("SELECT * FROM compra_juegos WHERE id_transaccion = %s", (t[0],))
        compra = mycursor.fetchall()
        cantidad = len(compra)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
    #juego
        if(len(compra)>0) :
            esjuego=True
            esDLC=False
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (compra[0][2],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
     #dlc
        if(len(compra)<=0) :
            print("***********************ha entrado en dlc***********************************")
            esDLC=True
            esjuego=False
            mycursor.execute("SELECT * FROM compra_dlcs WHERE id_transaccion = %s", (t[0],))
            compra = mycursor.fetchall()
            cantidad = len(compra)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (compra[0][2],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            
    #proveedor
        puntero_proveedor=compra[0][0]
        mycursor.execute("SELECT * FROM proveedor WHERE id_proveedor = %s", (puntero_proveedor,))
        provedor = mycursor.fetchone()
        pais= provedor[3]
        valoracion=provedor[1]
        
    #insertar datos en multidimensional    
        if esjuego:
            datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "juego":{"puntero_juego":puntero_juego,"genero":genero,"precio_juego":precio_juego,"clasificacion":clasificacion},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        elif esDLC:
             datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "dlc":{"puntero_dlc":puntero_dlc,"tipo":tipo,"precio_dlc":precio_dlc},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        print("---------------------------------------------------------------")        
        print(datos)
        x = colmul.insert_one(datos) 
        
    

Día de la semana: 0
2020-01-12 2017-01-01
---------------------------------------------------------------
{'movimiento': {'precio_final': 5, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '02', 'dia': '05'}, 'juego': {'puntero_juego': 7066, 'genero': ['adventure', 'casual'], 'precio_juego': 7, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 21, 'pais': 'Israel', 'valoracion': 3}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '06', 'dia': '07'}, 'juego': {'puntero_juego': 5826, 'genero': [], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 2, 'pais': 'Croatia', 'valoracion': 3}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 20, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '31'}, 'juego': {

---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '10', 'dia': '02'}, 'juego': {'puntero_juego': 5861, 'genero': ['action', 'adventure', 'RPG'], 'precio_juego': 6, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 3, 'pais': 'South Sudan', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '04', 'dia': '02'}, 'juego': {'puntero_juego': 12324, 'genero': ['casual', 'strategy', 'simulation'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 22, 'pais': 'Niger', 'valoracion': 4}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 6, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '03', 'dia': '30'}, 'juego

---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '02', 'dia': '06'}, 'juego': {'puntero_juego': 3741, 'genero': ['adventure', 'RPG'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 37, 'pais': 'Bahamas', 'valoracion': 7}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '06', 'dia': '16'}, 'juego': {'puntero_juego': 4448, 'genero': ['strategy'], 'precio_juego': 5, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 24, 'pais': 'Trinidad and Tobago', 'valoracion': 4}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 128, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '10', 'dia': '19'}, 'juego': {'puntero_juego': 110

---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '04', 'dia': '19'}, 'juego': {'puntero_juego': 9632, 'genero': ['casual'], 'precio_juego': 5, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 15, 'pais': 'Nepal', 'valoracion': 10}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '31'}, 'juego': {'puntero_juego': 3325, 'genero': ['action', 'adventure', 'strategy', 'RPG', 'early_access'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 14, 'pais': 'Virgin Islands, British', 'valoracion': 2}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 64, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '08', 'dia':

---------------------------------------------------------------
{'movimiento': {'precio_final': 0, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '03', 'dia': '01'}, 'juego': {'puntero_juego': 5631, 'genero': [], 'precio_juego': 0, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 26, 'pais': 'Laos', 'valoracion': 6}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 16, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '01'}, 'juego': {'puntero_juego': 11502, 'genero': ['action', 'adventure'], 'precio_juego': 20, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 25, 'pais': 'Ethiopia', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 3, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '04', 'dia': '20'}, 'juego': {'puntero_juego': 2518, 'genero': ['action',

...

---------------------------------------------------------------
{'movimiento': {'precio_final': 23, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '05', 'dia': '02'}, 'dlc': {'puntero_dlc': 220, 'tipo': ' cosmetico ', 'precio_dlc': 29}, 'proveedor': {'puntero_proveedor': 21, 'pais': 'Israel', 'valoracion': 3}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 24, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '09', 'dia': '23'}, 'dlc': {'puntero_dlc': 42, 'tipo': ' trama adicional', 'precio_dlc': 30}, 'proveedor': {'puntero_proveedor': 14, 'pais': 'Virgin Islands, British', 'valoracion': 2}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 19, 'tipo_movimiento': 'compra', 'cantidad': 

---------------------------------------------------------------
{'movimiento': {'precio_final': 9, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '15'}, 'dlc': {'puntero_dlc': 425, 'tipo': ' trama adicional', 'precio_dlc': 12}, 'proveedor': {'puntero_proveedor': 12, 'pais': 'Algeria', 'valoracion': 3}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '07', 'dia': '17'}, 'dlc': {'puntero_dlc': 542, 'tipo': ' cosmetico ', 'precio_dlc': 5}, 'proveedor': {'puntero_proveedor': 12, 'pais': 'Algeria', 'valoracion': 3}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'any

***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 12, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '10', 'dia': '24'}, 'dlc': {'puntero_dlc': 399, 'tipo': ' trama adicional', 'precio_dlc': 15}, 'proveedor': {'puntero_proveedor': 27, 'pais': 'Sweden', 'valoracion': 9}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 7, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '09', 'dia': '18'}, 'dlc': {'puntero_dlc': 426, 'tipo': 'moneda virtual ', 'precio_dlc': 9}, 'proveedor': {'puntero_proveedor': 15, 'pais': 'Nepal', 'valoracion': 10}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': 

***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 22, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '11', 'dia': '06'}, 'dlc': {'puntero_dlc': 237, 'tipo': ' trama adicional', 'precio_dlc': 28}, 'proveedor': {'puntero_proveedor': 5, 'pais': 'Benin', 'valoracion': 6}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 21, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '08', 'dia': '07'}, 'dlc': {'puntero_dlc': 594, 'tipo': 'moneda virtual ', 'precio_dlc': 27}, 'proveedor': {'puntero_proveedor': 10, 'pais': 'Sudan', 'valoracion': 4}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {

***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 15, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '01', 'dia': '20'}, 'dlc': {'puntero_dlc': 195, 'tipo': ' trama adicional', 'precio_dlc': 19}, 'proveedor': {'puntero_proveedor': 25, 'pais': 'Ethiopia', 'valoracion': 8}}
***********************ha entrado en dlc***********************************
---------------------------------------------------------------
{'movimiento': {'precio_final': 7, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '05'}, 'dlc': {'puntero_dlc': 530, 'tipo': ' cosmetico ', 'precio_dlc': 9}, 'proveedor': {'puntero_proveedor': 25, 'pais': 'Ethiopia', 'valoracion': 8}}
